In [53]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from pypfopt import expected_returns, risk_models, objective_functions, EfficientFrontier, objective_functions, CLA
from scipy.optimize import minimize
from scipy.stats import norm

# Define portfolio weights from ensemble learning algorithm
weights = np.array([0.25, 0.3, 0.15, 0.3])

# Define assets and risk-free rate
tickers = ['AAPL', 'AMZN', 'GOOG', 'NFLX']
start_date = '2018-03-31'
end_date = '2022-02-15'
data = yf.download(tickers, start=start_date, end=end_date)
mu = expected_returns.mean_historical_return(data)
mu=mu['Adj Close'].values
Sigma = risk_models.sample_cov(data)
Sigma=Sigma.loc['Adj Close','Adj Close'].values
# Define constraints
bounds = [(0,1) for i in range(len(tickers))]
# Define optimization constraints
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
# Define risk management parameters
max_drawdown = 0.1
volatility_target = 0.15
cvar_alpha = 0.05
gamma = 1
alpha = 1
# Define optimization problem
def objective_function(w, mu, Sigma, gamma, alpha, max_drawdown, volatility_target, cvar_alpha):
    # Calculate portfolio returns and risk
    portfolio_return = np.dot(w, mu)
    portfolio_volatility = np.sqrt(np.dot(w.T, np.dot(Sigma, w)))
    # Calculate portfolio drawdown
    portfolio_drawdown = max_drawdown - np.max(np.maximum.accumulate(portfolio_return.cumsum()) - portfolio_return.cumsum())
    # Calculate portfolio CVaR
    def calculate_cvar(weights, mu, Sigma, alpha):
    # Calculate the portfolio return and standard deviation
        port_return = np.dot(mu.T, weights)
        port_std_dev = np.sqrt(np.dot(weights.T, np.dot(Sigma, weights)))
    
    # Calculate the VaR at the given alpha level
        z_alpha = norm.ppf(alpha)
        VaR = -(port_return - z_alpha * port_std_dev)
    
    # Calculate the CVaR at the given alpha level
        cvar = -((1 / alpha) * norm.pdf(z_alpha)) * port_std_dev + VaR
    
        return cvar
    portfolio_cvar = calculate_cvar(w, mu, Sigma, cvar_alpha)
    # Calculate penalty terms
    volatility_penalty = gamma * np.square(portfolio_volatility - volatility_target)
    drawdown_penalty = alpha * np.square(portfolio_drawdown)
    
    # Calculate objective function
    obj = -portfolio_return + volatility_penalty + drawdown_penalty + portfolio_cvar
    return obj

# Optimize portfolio with risk management
result = minimize(objective_function, weights, args=(mu, Sigma, gamma, alpha, max_drawdown, volatility_target, cvar_alpha), 
                  method='SLSQP', constraints=constraints, bounds=bounds)

# Extract optimized portfolio weights
optimized_weights = result.x
optimized_weights

[*********************100%***********************]  4 of 4 completed


array([1.00000000e+00, 2.77555756e-17, 0.00000000e+00, 0.00000000e+00])

In [59]:
# Create EfficientFrontier object
ef = EfficientFrontier(mu, Sigma, weight_bounds=bounds)

# Optimize for maximum Sharpe ratio
weights = ef.max_sharpe()

# Print final portfolio weights
print(weights.values())

odict_values([0.909565529052391, 0.0, 0.0904344709476089, 0.0])
